# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

In [1]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    pass

In [2]:
class MongoHandler():
    pass

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!



In [3]:
import sqlite3
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import json

import sqlite3
con = sqlite3.connect ('database.sqlite')
cur = con.cursor()

In [4]:
cur.execute("""SELECT * FROM Matches;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [5]:
cur.execute("""SELECT * 
               FROM Matches
               WHERE Season == '2011';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


# German League Teams Only

In [6]:
cur.execute("""SELECT * 
               FROM Matches
               WHERE Season == '2011'
               AND (Div != 'E0') ;""")
dfb = pd.DataFrame(cur.fetchall())
dfb.columns = [x[0] for x in cur.description]
dfb.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


# Premier League Teams Only


In [7]:
cur.execute("""SELECT * 
               FROM Matches
               WHERE Season == '2011'
               AND (Div = 'E0');""")
dfp = pd.DataFrame(cur.fetchall())
dfp.columns = [x[0] for x in cur.description]
dfp.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,44495,E0,2011,2011-08-13,Blackburn,Wolves,1,2,A
1,44496,E0,2011,2011-08-13,Fulham,Aston Villa,0,0,D
2,44497,E0,2011,2011-08-13,Liverpool,Sunderland,1,1,D
3,44498,E0,2011,2011-08-13,Newcastle,Arsenal,0,0,D
4,44499,E0,2011,2011-08-13,QPR,Bolton,0,4,A


# Home Goals in total in 2011 

In [8]:
cur.execute("""SELECT Match_ID, Div , Season, Hometeam , Sum(FTHG) AS H_Goals
               FROM Matches
               WHERE Season == '2011' AND (Div != 'E0')
               GROUP BY Hometeam;""")
dfh = pd.DataFrame(cur.fetchall())
dfh.columns = [x[0] for x in cur.description]
dfh.head()

,Match_ID,Div,Season,HomeTeam,H_Goals
0,1138,D2,2011,Aachen,15
1,1106,D1,2011,Augsburg,20
2,1122,D1,2011,Bayern Munich,49
3,1127,D2,2011,Bochum,23
4,1140,D2,2011,Braunschweig,21


# Away Goals in total in 2011


In [9]:
cur.execute("""SELECT Match_ID, Div , Season, Awayteam , Sum(FTAG) AS A_Goals
              FROM Matches
              WHERE Season == '2011' AND (Div != 'E0')
              GROUP BY Awayteam;""")
dfa = pd.DataFrame(cur.fetchall())
dfa.columns = [x[0] for x in cur.description]
dfa.head()

,Match_ID,Div,Season,AwayTeam,A_Goals
0,1534,D2,2011,Aachen,15
1,1619,D1,2011,Augsburg,16
2,1092,D1,2011,Bayern Munich,28
3,1347,D2,2011,Bochum,18
4,1568,D2,2011,Braunschweig,16


In [10]:
df1 = dfa["A_Goals"]+dfh["H_Goals"]

In [11]:
df2 = pd.merge(dfa,dfh, on = 'Div', how = 'left')

In [12]:
df2

,Match_ID_x,Div,Season_x,AwayTeam,A_Goals,Match_ID_y,Season_y,HomeTeam,H_Goals
0,1534,D2,2011,Aachen,15,1138,2011,Aachen,15
1,1534,D2,2011,Aachen,15,1127,2011,Bochum,23
2,1534,D2,2011,Aachen,15,1140,2011,Braunschweig,21
3,1534,D2,2011,Aachen,15,1133,2011,Cottbus,18
4,1534,D2,2011,Aachen,15,1144,2011,Dresden,30
...,...,...,...,...,...,...,...,...,...
643,1466,D1,2011,Wolfsburg,18,1092,2011,Nurnberg,22
644,1466,D1,2011,Wolfsburg,18,1100,2011,Schalke 04,47
645,1466,D1,2011,Wolfsburg,18,1093,2011,Stuttgart,36
646,1466,D1,2011,Wolfsburg,18,1099,2011,Werder Bremen,31


In [13]:
df3 = df2.copy()

In [27]:
df3.head()

,Match_ID_x,Div,Season_x,AwayTeam,A_Goals,Match_ID_y,Season_y,HomeTeam,H_Goals
0,1534,D2,2011,Aachen,15,1138,2011,Aachen,15
1,1534,D2,2011,Aachen,15,1127,2011,Bochum,23
2,1534,D2,2011,Aachen,15,1140,2011,Braunschweig,21
3,1534,D2,2011,Aachen,15,1133,2011,Cottbus,18
4,1534,D2,2011,Aachen,15,1144,2011,Dresden,30


In [33]:
df3 = df3[['Match_ID_x' , 'Div', 'Season_x' , 'HomeTeam' , 'H_Goals', 'A_Goals' ]]

In [34]:
df3.head()

,Match_ID_x,Div,Season_x,HomeTeam,H_Goals,A_Goals
0,1534,D2,2011,Aachen,15,15
1,1534,D2,2011,Bochum,23,15
2,1534,D2,2011,Braunschweig,21,15
3,1534,D2,2011,Cottbus,18,15
4,1534,D2,2011,Dresden,30,15


In [36]:
df3['Total_goals'] = df3.apply(lambda x : x['H_Goals'] + x['A_Goals'] , axis = 1)
df3.head()

,Match_ID_x,Div,Season_x,HomeTeam,H_Goals,A_Goals,Total_goals
0,1534,D2,2011,Aachen,15,15,30
1,1534,D2,2011,Bochum,23,15,38
2,1534,D2,2011,Braunschweig,21,15,36
3,1534,D2,2011,Cottbus,18,15,33
4,1534,D2,2011,Dresden,30,15,45


# 1) German League Total wins and Goals


In [14]:
cur.execute("""SELECT HomeTeam, 
               Div , FTR,   
               SUM(FTHG) AS Home_Goals,
               Count(case WHEN FTR = 'H' then 1 else NULL end) AS Home_Wins,
               COUNT(CASE WHEN FTR = 'D' THEN 1 ELSE NULL END) AS Home_Draws
               FROM Matches
               WHERE Season = 2011 and Div != 'E0' 
               GROUP BY HomeTeam;""")
dfgh = pd.DataFrame(cur.fetchall())
dfgh.columns = [x[0] for x in cur.description]
dfgh.head()


,HomeTeam,Div,FTR,Home_Goals,Home_Wins,Home_Draws
0,Aachen,D2,A,15,4,6
1,Augsburg,D1,A,20,6,7
2,Bayern Munich,D1,A,49,14,1
3,Bochum,D2,A,23,7,3
4,Braunschweig,D2,A,21,6,8


# 2) Added column(Losses) and Deleted Away matches

In [15]:
dfgh['Home_Losses'] = 17-dfgh.Home_Wins-dfgh.Home_Draws
index = dfgh[(dfgh.FTR == 'A')| (dfgh.FTR == 'D')].index
dfgh = dfgh.drop(index, axis = 0)
dfgh

,HomeTeam,Div,FTR,Home_Goals,Home_Wins,Home_Draws,Home_Losses
5,Cottbus,D2,H,18,4,8,5
6,Dortmund,D1,H,44,14,2,1
7,Dresden,D2,H,30,8,5,4
8,Duisburg,D2,H,23,8,2,7
9,Ein Frankfurt,D2,H,38,11,5,1
12,Fortuna Dusseldorf,D2,H,35,11,5,1
15,Greuther Furth,D2,H,46,14,1,2
17,Hannover,D1,H,31,10,7,0
21,Ingolstadt,D2,H,28,6,8,3
23,Karlsruhe,D2,H,23,8,3,6


# 3) German League Total Away wins and Away Goals

In [16]:
cur.execute("""SELECT AwayTeam, 
               Div , FTR ,   
               SUM(FTAG) AS Away_Goals,
               Count(case WHEN FTR = 'A' then 1 else NULL end) AS Away_Wins,
               COUNT(CASE WHEN FTR = 'D' THEN 1 ELSE NULL END) AS Away_Draws
               FROM Matches
               WHERE Season = 2011 and Div != 'E0' 
               GROUP BY AwayTeam;""")
dfga = pd.DataFrame(cur.fetchall())
dfga.columns = [x[0] for x in cur.description]
dfga.head(10)

,AwayTeam,Div,FTR,Away_Goals,Away_Wins,Away_Draws
0,Aachen,D2,H,15,2,7
1,Augsburg,D1,H,16,2,7
2,Bayern Munich,D1,A,28,9,3
3,Bochum,D2,H,18,3,4
4,Braunschweig,D2,H,16,4,7
5,Cottbus,D2,H,12,4,3
6,Dortmund,D1,D,36,11,4
7,Dresden,D2,D,20,4,4
8,Duisburg,D2,D,19,2,7
9,Ein Frankfurt,D2,A,38,9,3


In [17]:
dfga['Away_Losses'] = 17-dfga.Away_Wins-dfga.Away_Draws
index = dfga[(dfga.FTR == 'H')| (dfga.FTR == 'D')].index
dfga = dfga.drop(index, axis = 0)
dfga


,AwayTeam,Div,FTR,Away_Goals,Away_Wins,Away_Draws,Away_Losses
2,Bayern Munich,D1,A,28,9,3,5
9,Ein Frankfurt,D2,A,38,9,3,5
15,Greuther Furth,D2,A,27,6,9,2
21,Ingolstadt,D2,A,15,2,5,10
25,M'gladbach,D1,A,20,8,2,7
27,Munich 1860,D2,A,27,7,4,6
28,Nurnberg,D1,A,16,6,2,9
29,Paderborn,D2,A,23,8,4,5
31,St Pauli,D2,A,23,6,5,6
35,Wolfsburg,D1,A,18,3,3,11


In [18]:
dfga["Total_Goals"] = dfga["Away_Goals"]+dfgh["Home_Goals"]

In [19]:
dfga

,AwayTeam,Div,FTR,Away_Goals,Away_Wins,Away_Draws,Away_Losses,Total_Goals
2,Bayern Munich,D1,A,28,9,3,5,NaN
9,Ein Frankfurt,D2,A,38,9,3,5,76.0
15,Greuther Furth,D2,A,27,6,9,2,73.0
21,Ingolstadt,D2,A,15,2,5,10,43.0
25,M'gladbach,D1,A,20,8,2,7,49.0
27,Munich 1860,D2,A,27,7,4,6,NaN
28,Nurnberg,D1,A,16,6,2,9,NaN
29,Paderborn,D2,A,23,8,4,5,NaN
31,St Pauli,D2,A,23,6,5,6,59.0
35,Wolfsburg,D1,A,18,3,3,11,NaN


# English League Total wins and Goals

In [20]:
#df2.loc[(df2['Div'] == 'D1') & (df2['Div'] == 'D2')]

In [21]:
df2['Total_goals'] = df2.apply(lambda x : x['H_Goals'] + x['A_Goals'] , axis = 1)

In [22]:
df2.head()

,Match_ID_x,Div,Season_x,AwayTeam,A_Goals,Match_ID_y,Season_y,HomeTeam,H_Goals,Total_goals
0,1534,D2,2011,Aachen,15,1138,2011,Aachen,15,30
1,1534,D2,2011,Aachen,15,1127,2011,Bochum,23,38
2,1534,D2,2011,Aachen,15,1140,2011,Braunschweig,21,36
3,1534,D2,2011,Aachen,15,1133,2011,Cottbus,18,33
4,1534,D2,2011,Aachen,15,1144,2011,Dresden,30,45


In [23]:
cur.execute("""Select * 
               FROM Matches
               Where Season == '2011'
               AND ((Div = 'D1' OR Div = 'D2'))
              
;""")
df1 = pd.DataFrame(cur.fetchall())
df1.columns = [x[0] for x in cur.description]
df1.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [24]:
df1.shape

(612, 9)

In [25]:
def grouped_bar_plot(dataframe, name):
    barWidth = 0.25

    r1 = np.arange(len(dataframe))
    r2 = [x + barWidth for x in r1]
    r3 = [x + barWidth for x in r2]

    sns.set_style('darkgrid')

    plt.figure(figsize = (20,10))
    plt.bar(r1, dataframe['Total_Wins'], color='yello', width=barWidth, edgecolor='white', label='wins')
    plt.bar(r2, dataframe['Total_Losses'], color='blue', width=barWidth, edgecolor='white', label='losses')
    plt.bar(r3, dataframe['Total_Draws'], color='gray', width=barWidth, edgecolor='white', label='draws')

    plt.xlabel('{} Teams'.format(name), fontweight='bold')
    plt.ylabel('Games', fontweight = 'bold')
    plt.xticks([r + barWidth for r in range(len(dataframe))], dataframe['Team'], rotation = 45)
    plt.legend()
    plt.title('{} Results - 2011 Season'.format(name), fontweight='bold')
    plt.savefig('{}_Results_2011'.format(name))
    plt.show()

In [26]:
df1 = df_final[df_final.League == 'EPL']
df1.sort_values('Total_Wins', inplace = True, ascending = False)
grouped_bar_plot(EPL, 'EPL')

NameError: name 'df_final' is not defined

# Putting all the dates in a dictionary to be matched with Weather API


In [ ]:
Date = list(df1.Date.unique())
len(Date)

In [ ]:
resp = requests.get("https://api.darksky.net/forecast/45c2523a2abba56c9086bf1bd12e36e1/52.5200,13.4050")
resp.json()

In [ ]:
resp = requests.get("https://api.darksky.net/forecast/45c2523a2abba56c9086bf1bd12e36e1/52.5200,13.4050,2011-12-11T12:00:00?currently,daily")
result = resp.json()

In [ ]:

resp.json().keys()

In [ ]:
result.keys

In [ ]:
result['offset']

In [ ]:

result['daily'].keys()

In [ ]:
result['daily']['data'][0]['icon']

In [ ]:
for key in result['daily'].keys():
    print(key, type(result['daily'][key]))

In [ ]:
df = pd.DataFrame(result['daily']['data'])
df.head()

In [ ]:
result['daily']['data'][0]['summary']

In [ ]:
result['timezone']

In [ ]:
data